In [1]:
from itertools import product
from typing import Dict, List, Optional, Union

import numpy as np
import pandas as pd
import param
from pycaret.classification import (
    compare_models,
    create_model,
    get_config,
    predict_model,
    setup,
    tune_model,
)
from pycaret.utils import check_metric

from feature_selection.run_pycaret_setup import run_pycaret_setup


class FeatureSelection(param.Parameterized):

    # Class attributes
    model_class_to_name = {
        "RidgeClassifier": "ridge",
        "LogisticRegression": "lr",
        "LinearDiscriminantAnalysis": "lda",
        "GradientBoostingClassifier": "gbc",
        "QuadraticDiscriminantAnalysis": "qda",
        "LGBMClassifier": "lightgbm",
        "AdaBoostClassifier": "ada",
        "RandomForestClassifier": "rf",
        "ExtraTreesClassifier": "et",
        "GaussianNB": "nb",
        "DecisionTreeClassifier": "dt",
        "KNeighborsClassifier": "knn",
        "SGDClassifier": "svm",
        "CatBoostClassifier": "catboost",
        "SVC": "rbfsvm",
        "GaussianProcessClassifier": "gpc",
        "MLPClassifier": "mlp",
        "XGBClassifier": "xgboost",
    }

    metrics_list = ["Accuracy", "AUC", "Recall", "Precision", "F1", "Kappa", "MCC"]

    # Private class attributes
    _filter_metric = {
        "Accuracy": 0.5,
        "AUC": 0.5,
        "Recall": 0.6,
        "Precision": 0.6,
        "F1": 0.6,
        "Kappa": 0.1,
        "MCC": 0.1,
    }

    _setup_kwargs = dict(
        preprocess=True,
        train_size=0.75,
        # test_data=test_data,
        session_id=123,
        normalize=True,
        transformation=True,
        ignore_low_variance=True,
        remove_multicollinearity=False,
        multicollinearity_threshold=0.4,
        n_jobs=-1,
        use_gpu=False,
        profile=False,
        ignore_features=None,
        fold_strategy="timeseries",
        remove_perfect_collinearity=True,
        create_clusters=False,
        fold=4,
        feature_selection=False,
        # you can use this to keep the 95 % most relevant features (fat_sel_threshold)
        feature_selection_threshold=0.4,
        combine_rare_levels=False,
        rare_level_threshold=0.02,
        pca=False,
        pca_method="kernel",
        pca_components=30,
        polynomial_features=False,
        polynomial_degree=2,
        polynomial_threshold=0.01,
        trigonometry_features=False,
        remove_outliers=False,
        outliers_threshold=0.01,
        feature_ratio=False,
        feature_interaction=False,
        # Makes everything slow AF. use to find out possibly interesting features
        interaction_threshold=0.01,
        fix_imbalance=False,
        log_experiment=False,
        verbose=False,
        silent=True,
        experiment_name="lagstest",
        html=False,
    )

    _numerics = ["int16", "int32", "int64", "float16", "float32", "float64", "int", "float"]

    # Init values
    ## Feature selection parameters
    target = param.String("goal_2.5")
    number_features = param.Number(
        0.5,
        bounds=(0, 1),
        inclusive_bounds=(False, False),
        doc="Number of features (percentage) selected each iteration. Only the first nth "
        "features will be kept for the next iteration.",
    )
    target_features = param.Number(
        0.3,
        bounds=(0, None),
        inclusive_bounds=(False, True),
        doc="Final total number of features. The goal of the package is to reduce "
        "the incoming columns of the dataset to this 'target_features' number.",
    )
    ## Metric parameters
    filter_metrics = param.Dict(_filter_metric)
    ## Model setup and model optimization parameters
    numerics = param.List(_numerics)
    ignore_features = param.List(default=[], allow_None=True)
    setup_kwargs = param.Dict(_setup_kwargs)
    include = param.List(default=None, item_type=str, allow_None=True)
    exclude = param.List(["qda", "knn", "nb"], item_type=str)
    sort = param.String("AUC")
    number_models = param.Integer(10, bounds=(2, 13))
    top_models = param.List(default=None, allow_None=True)
    optimize = param.Boolean(False)
    opt_list = param.List(["Accuracy", "Precision", "Recall", "F1", "AUC"], item_type=str)
    ## Class selectors
    dataset = param.ClassSelector(class_=pd.DataFrame)
    dict_models = param.ClassSelector(class_=dict)
    tune_dict_models = param.ClassSelector(class_=dict)
    x_train = param.ClassSelector(class_=pd.DataFrame)
    x_df = param.DataFrame(pd.DataFrame())
    model_df = param.ClassSelector(class_=pd.DataFrame)
    model_tuned_df = param.ClassSelector(class_=pd.DataFrame)
    features_df = param.ClassSelector(class_=pd.DataFrame)

    def __init__(self, dataset: pd.DataFrame, **kwargs):
        # Copy of the incoming dataset
        dataset = dataset.copy()
        # Compute the upper bound of number_features, target_features, number_models
        total_features = dataset.shape[1]
        self.param.target_features.bounds = (0, total_features)
        if "include" in kwargs:
            self.param.number_models.default = len(kwargs["include"])
            self.param.number_models.bounds = (0, len(kwargs["include"]))
        # Call super
        super(FeatureSelection, self).__init__(dataset=dataset, **kwargs)
        # Get the features of the dataframe
        self.feature_list = self.dataset.columns.tolist()
        self.feature_list.remove(self.target)  # target column should not be counted
        # Compute target features
        self.target_features = self.calculate_number_features(
            number_features=self.target_features, features=self.feature_list
        )
        # Get the evaluator and the arguments. Depends on the "include" parameter
        self._training_function, self._args = self._decide_model_eval()
        # Get all the columns whose type is numeric
        self.numeric_features = self._compute_numeric_features(df=self.dataset[self.feature_list])

    def _compute_numeric_features(self, df: pd.DataFrame):
        """Return those columns from the given dataset whose data type is numeric."""
        return df.select_dtypes(include=self.numerics).columns.tolist()

    def _decide_model_eval(self):
        """
        Define the pycaret model evaluator depending on the number of included models.

        If the 'include' list parameter equals 1, the method will return
        the 'create_models' pycaret object.
        If 'include' parameter list is greatear than 1, the method will
        return the 'compare_model' pycaret object and its arguments.
        If 'include' parameter equals None, the method will return the
        'compare_models' pycaret object, where all possible models are
        considered for evaluation, except those included within the 'exclude'
        list.
        """
        args = {"n_select": self.number_models, "sort": self.sort, "verbose": False}
        training_function = compare_models
        if not self.include:
            args["exclude"] = self.exclude
        elif len(self.include) == 1:
            training_function = lambda *rgs, **kwargs: [create_model(*rgs, **kwargs)]
            args = {"estimator": self.include[0], "verbose": False}
        else:
            args["include"] = self.include
        return training_function, args

    @staticmethod
    def calculate_number_features(
        number_features: Union[int, float], features: Union[pd.DataFrame, List]
    ) -> int:
        n_features = (
            int(number_features)
            if (number_features >= 1)
            else int(number_features * len(features))
        )
        return n_features

    def train_model(self):
        """Preprocess the data and select self.number_models top models."""
        # Selected dataset
        selected_cols = self.feature_list + [self.target]
        train_data = self.dataset[selected_cols] if self.x_df.empty else self.x_df[selected_cols]
        # Numeric features
        self.setup_kwargs["numeric_features"] = [
            c for c in self.numeric_features if c in self.feature_list
        ]
        # Ignore features
        self.setup_kwargs["ignore_features"] = [
            c for c in self.ignore_features if c in self.feature_list
        ]
        # Initialize pycaret setup
        setup(data=train_data, target=self.target, **self.setup_kwargs)
        # Get train dataset and preprocessed dataframe
        self.x_train = get_config("X_train")
        if self.x_df.empty:  # TODO change x_df by dataset and add flag?
            self.x_df = pd.concat([get_config("X"), get_config("y")], axis=1)
            self.setup_kwargs["preprocess"] = False  # Turn off preprocessing
        # Compare models
        self.top_models = self._training_function(**self._args)

    def create_dict_models(self):
        """Create a dictionary whose values are pycaret standard models."""
        self.dict_models = {
            str(top_model).split("(")[0]: top_model for top_model in self.top_models
        }
        # Remove bad catboost key
        oldkey = [key for key in self.dict_models.keys() if key.startswith("<catboost")]
        if oldkey:
            self.dict_models["CatBoostClassifier"] = self.dict_models.pop(oldkey[0])
        # Remap
        self.dict_models = {
            self.model_class_to_name[key]: self.dict_models[key] for key in self.dict_models.keys()
        }

    def create_dict_tuned_models(self):
        """Create a dictionary whose keys and values are pycaret tuned models."""
        self.tune_dict_models = {}
        for (model_str, py_model), optimize in product(self.dict_models.items(), self.opt_list):
            self.tune_dict_models[f"{model_str}_tune_{optimize}"] = tune_model(
                py_model,
                optimize=optimize,
                verbose=False,
                n_iter=30,
                choose_better=True,
            )

    def get_metrics_df(self, test_predicted, model, dataframe):
        """Compute different metric values for the given model."""
        value_dct = dict()
        for metric in self.metrics_list:
            try:
                value_dct[metric] = check_metric(
                    actual=test_predicted[self.target],
                    prediction=test_predicted["Label"],
                    metric=metric,
                )
            except AttributeError:
                value_dct[metric] = np.nan
        for key, val in value_dct.items():
            dataframe.loc[model, key] = val
        return dataframe

    def remove_bad_models(self, dataframe: pd.DataFrame):
        """Filter and remove the models whose metrics do not satisfy the given conditions."""
        remove_dict = dict()
        models = dataframe.index.tolist()
        for model, (metric, cond) in product(models, self.filter_metrics.items()):
            if dataframe.loc[model, metric] < cond:
                remove_dict[model] = metric
        remove_models = list(set(remove_dict.keys()))
        dataframe.drop(labels=remove_models, axis="index", inplace=True)
        return dataframe

    def filter_best_features(self, key_model: str, models_dict: Dict):
        """Compute the most relevant features used by the given model."""
        py_model = models_dict[key_model]
        cond = any([key_model.startswith(name) for name in ["lr", "lda", "ridge", "svm"]])
        score_metric = abs(py_model.coef_[0]) if cond else py_model.feature_importances_
        metrics_dict = {
            "model_id": key_model,
            "model": key_model.split("_")[0],
            "feature": self.x_train.columns,
            "score": score_metric,
        }
        df = pd.DataFrame(metrics_dict).sort_values(by="score", ascending=False)
        top_n_features = self.calculate_number_features(
            number_features=self.number_features,
            features=df,
        )
        return df.iloc[:top_n_features]

    def extract_features(self, dataframe: pd.DataFrame, dict_models: Dict):
        """Update self.features_df with the most relevant features used by the given model."""
        models = dataframe.index.tolist()
        for model in models:  # model extracted from dataframe
            # Check
            if (
                model not in dict_models.keys()
            ):  # check no errors have been produced during operations
                raise KeyError(f"The selected model: {model} is not listed in dict_models.keys()")
            df_conc = self.filter_best_features(key_model=model, models_dict=dict_models)
            self.features_df = pd.concat([self.features_df, df_conc])

    def compute_metrics_df(self):
        """Update self.features_df with the most relevant features used by the standard models."""
        self.model_df = pd.DataFrame(
            data=[], index=self.dict_models.keys(), columns=self.metrics_list
        )
        for model, py_model in self.dict_models.items():
            predict = predict_model(py_model)
            self.model_df = self.get_metrics_df(
                test_predicted=predict,
                model=model,
                dataframe=self.model_df,
            )
        self.model_df = self.remove_bad_models(dataframe=self.model_df)
        self.extract_features(dataframe=self.model_df, dict_models=self.dict_models)

    def filter_tuned_duplicate(self):
        """Remove tuned models with identical metrics."""

        def drop(x):
            x.drop_duplicates(subset=self.metrics_list, keep="first", inplace=True)
            return x

        df = self.model_tuned_df.groupby("model").apply(drop)
        df = (
            df.drop(columns="model").reset_index(level="model")
            if isinstance(df.index, pd.MultiIndex)
            else df
        )
        return df

    def tune_df(self):
        """Update self.features_df with the most relevant features used by tuned models."""
        # Tune dataframe
        self.model_tuned_df = pd.DataFrame(
            data=[],
            index=self.tune_dict_models.keys(),
            columns=["model"] + self.metrics_list,
        )
        # Model entry
        for prim_model in self.dict_models.keys():
            ix = [ind.startswith(prim_model) for ind in self.model_tuned_df.index]
            self.model_tuned_df.loc[ix, "model"] = prim_model
        # Fill dataframe
        for model, py_model in self.tune_dict_models.items():
            predict = predict_model(py_model)
            self.model_tuned_df = self.get_metrics_df(
                test_predicted=predict, model=model, dataframe=self.model_tuned_df
            )
        # Remove duplicate and filter
        self.model_tuned_df = self.filter_tuned_duplicate()
        self.model_tuned_df = self.remove_bad_models(dataframe=self.model_tuned_df)
        # Get features
        self.extract_features(dataframe=self.model_tuned_df, dict_models=self.tune_dict_models)

    def run_feature_extraction(self):
        """Update self.features_df with the most relevant features used by each model."""
        # Initialize feature dataframe and train model
        self.features_df = pd.DataFrame(data=[], columns=["model_id", "model", "feature", "score"])
        self.train_model()
        # Run standard models
        self.create_dict_models()
        self.compute_metrics_df()
        # Run tuned models
        if self.optimize:
            self.create_dict_tuned_models()
            if not bool(self.tune_dict_models):
                raise ValueError("The tune dictionary is empty!")
            self.tune_df()
        # Return the list containing the features and their score
        self.features_df.index.name = "index_rem"
        return self.features_df.reset_index().drop(columns="index_rem")

    def remove_zeros(self):
        """Remove non-relevant features (those with a zero score)."""
        ix = self.features_df["score"] <= 0
        self.features_df.drop(index=self.features_df.loc[ix].index, inplace=True)
        self.features_df.reset_index(drop=True, inplace=True)

    @staticmethod
    def normalize(dataframe):
        """Normalize the pycaret score of each feature."""

        def norm(x):
            x["normal"] = x["score"] / x["score"].max()
            return x

        return (
            dataframe.groupby("model_id")
            .apply(norm)
            .sort_values(["model_id", "normal"], ascending=[True, False])
        )

    @staticmethod
    def feature_score(dataframe):
        """Assign the score to the selected features."""
        group = dataframe.groupby("feature")
        sorted_data = group.agg(
            counts=pd.NamedAgg(column="normal", aggfunc="count"),
            normal_sum=pd.NamedAgg(column="normal", aggfunc="sum"),
        ).sort_values(by=["counts", "normal_sum"], ascending=False)
        sorted_data["final_score"] = sorted_data["normal_sum"] / sorted_data["counts"]
        return sorted_data.sort_values("final_score", ascending=False)

    def create_feature_list(self):
        """Run all necessary methods to extract the list of relevant features."""
        # Call creation features dataframe
        self.features_df = self.run_feature_extraction()
        # Remove zeros and normalize
        self.remove_zeros()
        self.features_df = self.normalize(dataframe=self.features_df)
        # Get score
        scoreboard = self.feature_score(dataframe=self.features_df)
        top_n_features = self.calculate_number_features(
            number_features=self.number_features,
            features=scoreboard,
        )
        filtered = scoreboard.iloc[:top_n_features]
        self.feature_list = filtered.index.tolist()

    def repeat_pipeline(self):
        """Iterate over the process to create the feature list and repeat it self.repeat times."""
        while len(self.feature_list) > self.target_features:
            # Call iteration
            self.create_feature_list()
            if len(self.feature_list) <= 1:
                break
        return self.feature_list

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

from ml_bets.constants import FEATURES_PATH
from ml_bets.features.features import Features
from ml_bets.modeling.match_model import PipelineDatasets, run_pycaret_setup


from pycaret.utils import check_metric
from pycaret.classification import (add_metric, calibrate_model, optimize_threshold,
    create_model,
    finalize_model,
    optimize_threshold,              
    save_model,
    compare_models, 
    evaluate_model,
    get_config,
    setup,
    tune_model,
    predict_model,
)

from ml_bets.supplementary.functions import IGNORE_FEATURES
from ml_bets.research.datasets import create_dataset
from ml_bets.features.names.goals import GOALS_FEATURES, NEW_FEATURES

In [3]:
target = "goals_2.5"
test_date = "1-Dec-2021"

In [4]:
feats = Features()

c:\users\usuario\desktop\data science\ml_bets-master\data\future_matches\ESP1C.xls
c:\users\usuario\desktop\data science\ml_bets-master\data\future_matches\ING1C.xls
c:\users\usuario\desktop\data science\ml_bets-master\data\future_matches\ITA1C.xls
Excel file c:\users\usuario\desktop\data science\ml_bets-master\data\future_matches\MEX1C.xls is empty. Skipping
c:\users\usuario\desktop\data science\ml_bets-master\data\future_matches\FRA1C.xls
Excel file c:\users\usuario\desktop\data science\ml_bets-master\data\future_matches\MLS1C.xls is empty. Skipping
c:\users\usuario\desktop\data science\ml_bets-master\data\future_matches\ALE1C.xls


In [5]:
def patched_create_dataset(
    test_date: str,
    target: str,
    columns=None,
    odds_features: bool = True,
    summary: bool = True,
    odds_rankings: bool = True,
    include_std: bool = True,
    features: Features = None,
    ignore_features=None,
    drop_future_matches: bool = True,
    test_weeks: int = 4,
):
    features = features or Features()
    examples = features.create(
        columns=columns,
        odds_features=odds_features,
        odds_rankings=odds_rankings,
        referee_features=True,
        include_std=include_std,
        summary=summary,
    )
    examples = examples[[x for x in examples.columns if "possession" not in x]]
    if ignore_features is not None:
        examples.drop(columns=ignore_features, inplace=True)
    pds = PipelineDatasets(
        examples=examples,
        features=features,
        target=target,
        drop_future_matches=drop_future_matches,
        test_size=test_date,
        test_weeks=test_weeks,
    )
    return pds

In [6]:
ds = patched_create_dataset(target=target,
                    test_date=test_date,
                    features=feats,
                    odds_features=True,
                    include_std=True,
                    ignore_features=IGNORE_FEATURES+["referee", "hour_rank", "hour_before_16", "is_weekend"],
                    drop_future_matches=False,
                    
                   )

In [7]:
train_data = ds.train_data.copy()


In [8]:
setup_kwargs = dict(
        preprocess=True,
        #custom_pipeline=loaded,
        train_size=0.75,
        session_id=123,
        normalize=True,
       # normalize_method="robust",
        transformation=True,
        ignore_low_variance=True,
        remove_multicollinearity=False,
        multicollinearity_threshold=0.8,
        n_jobs=-1,
        use_gpu=False,
        profile=False,
        #ignore_features=ignore_features,
        fold_strategy="stratifiedkfold",#"timeseries",
        remove_perfect_collinearity=True,
        create_clusters=False,
        fold=3,
        feature_selection=False,
        # you can use this to keep the 95 % most relevant features (fat_sel_threshold)
        feature_selection_threshold=0.5,
        combine_rare_levels=False,
        rare_level_threshold=0.02,
        pca=False,
        pca_method="kernel",
        pca_components=50,
        polynomial_features=False,
        polynomial_degree=2,
        polynomial_threshold=0.05,
        trigonometry_features=False,
        remove_outliers=True,
        outliers_threshold=0.01,
        feature_ratio=False,
        feature_interaction=False,
        # Makes everything slow AF. use to find out possibly interesting features
        interaction_threshold=0.05,
        fix_imbalance=True,
        log_experiment=False,
        verbose=False,
        silent=True,
        experiment_name="lagstest",
    )

## Feature selection (without triplet)

In [8]:
metric_param = {
        "Accuracy": -0.1,
        "AUC": -0.1,
        "Recall": -0.1,
        "Precision": -0.1,
        "F1": -0.1,
        "Kappa": -1.0,
        "MCC": -1.0,
    }
feat_sel = FeatureSelection(target=target,
                            dataset=train_data.dropna(),#[list(set(new_subset+new_feat+[target]))],
                            target_features=500,
                            filter_metrics=metric_param,
                            include=["lr"],
                            setup_kwargs=setup_kwargs,
                            optimize=True,
                            opt_list=["AUC"],
                           )
selected_features = feat_sel.repeat_pipeline()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5158,0.5292,0.4973,0.5257,0.5111,0.0323,0.0323


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5103,0.5309,0.4946,0.5199,0.5069,0.0212,0.0212


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5282,0.5543,0.5351,0.5366,0.5359,0.0561,0.0561


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5296,0.5570,0.5297,0.5385,0.5341,0.0591,0.0591


In [9]:
sorted(selected_features)

['DEF_mean_a',
 'MED_diff',
 'accurate_cross_nocorner_std_diff',
 'accurate_cross_std_h',
 'accurate_fwd_zone_pass_mean_diff',
 'accurate_fwd_zone_pass_std_a',
 'accurate_goal_kicks_std_h',
 'accurate_keeper_sweeper_mean_a',
 'accurate_keeper_sweeper_mean_h',
 'accurate_keeper_sweeper_std_a',
 'accurate_launches_mean_a',
 'accurate_layoffs_mean_a',
 'accurate_layoffs_std_a',
 'accurate_pass_std_diff',
 'accurate_through_ball_mean_diff',
 'accurate_throws_std_diff',
 'accurate_throws_std_h',
 'att_assist_openplay_mean_a',
 'att_corner_ratio_mean_a',
 'att_freekick_goal_std_diff',
 'att_freekick_miss_mean_h',
 'att_freekick_miss_std_a',
 'att_freekick_total_mean_a',
 'att_freekick_total_mean_h',
 'att_goal_high_centre_mean_diff',
 'att_goal_high_centre_std_diff',
 'att_goal_high_left_mean_diff',
 'att_goal_high_left_std_h',
 'att_goal_high_right_std_a',
 'att_goal_high_right_std_diff',
 'att_goal_low_centre_mean_a',
 'att_goal_low_centre_std_a',
 'att_goal_low_centre_std_h',
 'att_goal_l

#### new imports 

In [8]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

from ml_bets.constants import FEATURES_PATH
from ml_bets.features.features import Features
from ml_bets.modeling.match_model import PipelineDatasets, run_pycaret_setup
#from featsel.feature_selection import FeatureSelection

from pycaret.utils import check_metric
from pycaret.classification import (add_metric, calibrate_model, optimize_threshold,
    create_model,
    finalize_model,
    optimize_threshold,              
    save_model,
    compare_models, 
    evaluate_model,
    get_config,
    setup,
    tune_model,
    predict_model,
)

from ml_bets.supplementary.functions import IGNORE_FEATURES
from ml_bets.research.datasets import create_dataset
from ml_bets.features.names.goals import GOALS_FEATURES, NEW_FEATURES

In [9]:
from ml_bets.research.tips import calibrate_tips, tips_from_model, combine_tips, predict_dataset, compose_tips, get_tip_probs

In [10]:
ix = np.logical_and(feats.matches["date"].dt.month > 10,
                    feats.matches["competition"].isin({"mexican_primera", 'us_major_league_soccer'}))
index = feats.matches[~ix].index

In [11]:
def setup_dataset(test_date):
    ds = create_dataset(target=target,
                        test_date=test_date,
                        features=feats,
                        odds_features=True,
                        include_std=True,
                        test_weeks=4,
                        ignore_features=IGNORE_FEATURES,
                        drop_future_matches=False,
                       )
    train_data = ds.train_data.copy()#[ds.train_data.index.map(lambda x: "us_major_league_soccer" not in x and "mexican" not in x)]
    train_data.drop(columns=["hour_rank", "hour_before_16", "is_weekend"], inplace=True)
    train_data = train_data[train_data.index.isin(index)][list(set(selected_features)) + [target]].copy()#.reset_index(drop=True)
    test_data = ds.test_set[ds.test_set.index.isin(index)][list(set(selected_features)) + [target]].copy()#.reset_index(drop=True)
    val_data = ds.val_set[ds.val_set.index.isin(index)].copy()
    setup_kwargs = dict(
        preprocess=True,
        test_data=test_data[train_data.columns.tolist()],#.dropna(),
        #numeric_features=[x for x in train_data.columns.tolist() if x != target],
        #custom_pipeline=loaded,
        #train_size=0.75,
        session_id=123,
        normalize=True,
        normalize_method="robust",
        transformation=True,
        ignore_low_variance=True,
        remove_multicollinearity=False,
        multicollinearity_threshold=0.8,
        n_jobs=-1,
        use_gpu=False,
        profile=False,
        #ignore_features=ignore_features,
        fold_strategy="stratifiedkfold",#"timeseries",
        remove_perfect_collinearity=True,
        create_clusters=False,
        fold=4,
        feature_selection=False,
        # you can use this to keep the 95 % most relevant features (fat_sel_threshold)
        feature_selection_threshold=0.5,
        combine_rare_levels=False,
        rare_level_threshold=0.02,
        pca=True,
        pca_method="kernel",
        pca_components=50,
        polynomial_features=False,
        polynomial_degree=2,
        polynomial_threshold=0.05,
        trigonometry_features=False,
        remove_outliers=True,
        outliers_threshold=0.01,
        feature_ratio=False,
        feature_interaction=False,
        # Makes everything slow AF. use to find out possibly interesting features
        interaction_threshold=0.05,
        fix_imbalance=True,
        log_experiment=False,
        verbose=False,
        silent=True,
        experiment_name="lagstest",
    )
    _ = setup(data=train_data, target=target, **setup_kwargs)
    return train_data, test_data, val_data, ds

In [37]:
train_data, test_data, val_data, ds = setup_dataset("4-Dec-2021")

In [38]:
from pycaret.classification import stack_models, ensemble_model, blend_models
def train_ensemble():
    top_models = compare_models(
            n_select=8,
            sort='MCC',
            include=["lr", "lda", "ridge", "et", "rf", "svm"],
            verbose=True,
        )
    tuned_models = [tune_model(model, optimize="MCC", choose_better=True, n_iter=50, search_library="optuna") for model in top_models]
    cali = [calibrate_model(tuned, method="sigmoid", calibrate_fold=4) for tuned in tuned_models]
    blend = blend_models(cali)
    opti = tune_model(blend, optimize="Precision", choose_better=True, n_iter=50, search_library="optuna")
    return opti, cali, tuned_models

In [39]:
top_models = compare_models(
            n_select=8,
            sort='MCC',
            include=["lr", "lda", "ridge", "svm"],
            verbose=True,
        )

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.5831,0.6161,0.5937,0.6030,0.5971,0.1652,0.1658,0.6300
svm,SVM - Linear Kernel,0.5780,0.0000,0.5975,0.6037,0.5845,0.1544,0.1656,0.2050
ridge,Ridge Classifier,0.5825,0.0000,0.5937,0.6018,0.5966,0.1639,0.1645,0.2000
lda,Linear Discriminant Analysis,0.5819,0.6107,0.5962,0.6003,0.5972,0.1624,0.1629,0.2100


In [41]:
evaluate_model(top_models[0])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

## Feature selection (adding triplet)

In [10]:
common_feat = list(set(["_".join(x.split("_")[:-1]) for x in selected_features]))
new_subset = [x for x in train_data.columns for c in common_feat if c in x]

len(selected_features), len(new_subset), sorted(new_subset)

(181,
 583,
 ['ATT_h_mul_shots_mul_goals_mean_diff',
  'DEF_h_mul_shots_mul_goals_mean_diff',
  'DEF_mean_a',
  'DEF_mean_diff',
  'DEF_mean_h',
  'MED_a',
  'MED_diff',
  'MED_h',
  'MED_mean_a',
  'MED_mean_diff',
  'MED_mean_h',
  'MED_std_a',
  'MED_std_diff',
  'MED_std_h',
  'accurate_cross_nocorner_std_a',
  'accurate_cross_nocorner_std_diff',
  'accurate_cross_nocorner_std_h',
  'accurate_cross_std_a',
  'accurate_cross_std_diff',
  'accurate_cross_std_h',
  'accurate_fwd_zone_pass_mean_a',
  'accurate_fwd_zone_pass_mean_diff',
  'accurate_fwd_zone_pass_mean_h',
  'accurate_fwd_zone_pass_std_a',
  'accurate_fwd_zone_pass_std_diff',
  'accurate_fwd_zone_pass_std_h',
  'accurate_goal_kicks_std_a',
  'accurate_goal_kicks_std_diff',
  'accurate_goal_kicks_std_h',
  'accurate_keeper_sweeper_mean_a',
  'accurate_keeper_sweeper_mean_diff',
  'accurate_keeper_sweeper_mean_h',
  'accurate_keeper_sweeper_std_a',
  'accurate_keeper_sweeper_std_diff',
  'accurate_keeper_sweeper_std_h',
  '

In [12]:
"""Monkey patch feature_selection module."""
from typing import List, Tuple

# from feature_selection import FeatureSelection


def get_root_features(feature_list: List) -> Tuple[List, List]:
    """Get the features related to teams, i.e., those ended with "_a", "_h", "_diff"."""
    # Conditions
    cond1 = lambda x: x.endswith("_h")
    cond2 = lambda x: x.endswith("_a")
    cond3 = lambda x: x.endswith("_diff")
    # Features ended with _h, _a, _diff
    sublist = [c for c in feature_list if (cond1(c) or cond2(c) or cond3(c))]
    return sublist, list(set(["_".join(c.split("_")[:-1]) for c in sublist]))


def append_list(root_features: List, og_columns: List):
    """Append suffixes to the incoming feature labels."""
    # Create empty list
    dressed_list = list()
    # Add  suffixes to each root label
    # dressed_list = [c for root in root_features for c in df_columns if c.startswith(root)]
    for c in root_features:
        dressed_list.append(f"{c}_h")
        dressed_list.append(f"{c}_a")
        dressed_list.append(f"{c}_diff")
    dressed_list = [c for c in dressed_list if c in og_columns]
    # return list(set(dressed_list))
    return dressed_list


def massage_feat_list(feature_list: List, og_columns: List):
    """
    Add team features to the incoming list.

    Given a feature list, this function adds the full trio of team
    features (*_h, *_a, *_diff) to those already present that are
    related to team statistics.
    """
    feature_list = feature_list.copy()
    # Get list of match features and root
    sublist, root = get_root_features(feature_list)
    # Get features not related to teams
    no_team = list(set(feature_list) - set(sublist))
    # Add suffixes to roots
    dressed = append_list(root_features=root, og_columns=og_columns.copy())
    # return list(set(no_team + dressed))
    return no_team + dressed


def finalize_list(feature_list: List, df_columns: List):
    """
    Complete the feature_list returned by the selection module.

    Once the number of features has been reduced to a subset of
    relevant features, this function completes the list by adding
    team features to those already present.

    If a team feature is present, but the set only contains the one
    related to the home or away team, the function adds the corresponding
    feature related to the difference between teams, i.e., team_feat_diff.
    """
    feature_list = feature_list.copy()
    # Elements with _h, _a, _diff
    sublist, roots = get_root_features(feature_list)
    # List to be used as building block
    final_list = list(set(feature_list) - set(sublist))
    # Check for team features
    for root in roots:
        triplet = [f"{root}_h", f"{root}_a", f"{root}_diff"]
        subset = [c for tri in triplet for c in sublist if c == tri]
        if len(subset) < 2 and not subset[0].endswith("_diff"):
            subset.append(f"{root}_diff")
        final_list = final_list + subset
    final_list = [c for c in final_list if c in df_columns]
    return final_list


def patch_repeat_pipeline(self):
    """Iterate over the process to create the feature list and repeat it self.repeat times (PATCH)."""
    i = 0
    while len(self.feature_list) > self.target_features:
        i += 1
        print(f"{i} iteration")
        # Call iteration
        self.create_feature_list()
        self.feature_list = massage_feat_list(
            feature_list=self.feature_list, og_columns=self.x_df.columns
        )
        if len(self.feature_list) <= 1:
            break
    return finalize_list(feature_list=self.feature_list, df_columns=self.x_df.columns)

def patch_train_model(self):
        """Preprocess the data and select self.number_models top models."""
        # Selected dataset
        selected_cols = self.feature_list + [self.target]
        train_data = self.dataset[selected_cols] if self.x_df.empty else self.x_df[selected_cols]
        # Numeric features
        self.setup_kwargs["numeric_features"] = [
            c for c in self.numeric_features if c in self.feature_list
        ]
        # Ignore features
        self.setup_kwargs["ignore_features"] = [
            c for c in self.ignore_features if c in self.feature_list
        ]
        self.setup_kwargs["feature_interaction"]: False
        # Initialize pycaret setup
        setup(data=train_data, target=self.target, **self.setup_kwargs)
        # Get train dataset and preprocessed dataframe
        self.x_train = get_config("X_train")
        if self.x_df.empty:  # TODO change x_df by dataset and add flag?
            self.x_df = pd.concat([get_config("X"), get_config("y")], axis=1)
            self.setup_kwargs["preprocess"] = False  # Turn off preprocessing
        # Compare models
        self.top_models = self._training_function(**self._args)

In [13]:
FeatureSelection.train_model = patch_train_model

In [14]:
FeatureSelection.repeat_pipeline = patch_repeat_pipeline

In [16]:
metric_param = {
        "Accuracy": 0.1,
        "AUC": 0.1,
        "Recall": 0.1,
        "Precision": 0.1,
        "F1": 0.1,
        "Kappa": -1.0,
        "MCC": -1.0,
    }
feat_sel = FeatureSelection(target=target,
                            dataset=train_data.dropna(),#[list(set(new_subset+new_feat+[target]))],
                            target_features=500,
                            filter_metrics=metric_param,
                            include=["lr"],
                            setup_kwargs=setup_kwargs,
                            optimize=True,
                            opt_list=["AUC" , "Accuracy" , "Precision" , "Recall"],
                            number_features = 0.56
                           )
triplet_selected_features = feat_sel.repeat_pipeline()




1 iteration


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5186,0.5299,0.5000,0.5286,0.5139,0.0378,0.0378


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5089,0.5322,0.4892,0.5186,0.5035,0.0186,0.0186


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5089,0.5322,0.4892,0.5186,0.5035,0.0186,0.0186


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5089,0.5322,0.4892,0.5186,0.5035,0.0186,0.0186


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5543,0.5732,0.5703,0.5612,0.5657,0.1081,0.1081


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5488,0.5709,0.5595,0.5565,0.5580,0.0973,0.0973


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5488,0.5709,0.5595,0.5565,0.5580,0.0973,0.0973


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5488,0.5709,0.5595,0.5565,0.5580,0.0973,0.0973


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5502,0.5699,0.5676,0.5570,0.5622,0.0998,0.0998


In [18]:
len(triplet_selected_features
   )

456

In [13]:
common_feat = list(set(["_".join(x.split("_")[:-1]) for x in selected_features]))
new_subset = [x for x in train_data.columns for c in common_feat if c in x]

len(selected_features), len(new_subset), sorted(new_subset)

(145,
 276,
 ['ATT_h_mul_shots_mul_goals_mean_diff',
  'ATT_h_mul_shots_mul_goals_mean_diff',
  'DEF_h_mul_shots_mul_goals_mean_diff',
  'DEF_h_mul_shots_mul_goals_mean_diff',
  'accurate_cross_nocorner_std_a',
  'accurate_cross_nocorner_std_diff',
  'accurate_cross_nocorner_std_h',
  'accurate_cross_std_a',
  'accurate_cross_std_diff',
  'accurate_cross_std_h',
  'accurate_goal_kicks_std_a',
  'accurate_goal_kicks_std_diff',
  'accurate_goal_kicks_std_h',
  'accurate_keeper_sweeper_mean_a',
  'accurate_keeper_sweeper_mean_diff',
  'accurate_keeper_sweeper_mean_h',
  'accurate_keeper_sweeper_std_a',
  'accurate_keeper_sweeper_std_diff',
  'accurate_keeper_sweeper_std_h',
  'accurate_launches_mean_a',
  'accurate_launches_mean_diff',
  'accurate_launches_mean_h',
  'accurate_layoffs_mean_a',
  'accurate_layoffs_mean_diff',
  'accurate_layoffs_mean_h',
  'accurate_layoffs_std_a',
  'accurate_layoffs_std_diff',
  'accurate_layoffs_std_h',
  'accurate_through_ball_mean_a',
  'accurate_thro

#### new imports 

In [12]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

from ml_bets.constants import FEATURES_PATH
from ml_bets.features.features import Features
from ml_bets.modeling.match_model import PipelineDatasets, run_pycaret_setup
#from featsel.feature_selection import FeatureSelection

from pycaret.utils import check_metric
from pycaret.classification import (add_metric, calibrate_model, optimize_threshold,
    create_model,
    finalize_model,
    optimize_threshold,              
    save_model,
    compare_models, 
    evaluate_model,
    get_config,
    setup,
    tune_model,
    predict_model,
)

from ml_bets.supplementary.functions import IGNORE_FEATURES
from ml_bets.research.datasets import create_dataset
from ml_bets.features.names.goals import GOALS_FEATURES, NEW_FEATURES

In [13]:
from ml_bets.research.tips import calibrate_tips, tips_from_model, combine_tips, predict_dataset, compose_tips, get_tip_probs

In [14]:
ix = np.logical_and(feats.matches["date"].dt.month > 10,
                    feats.matches["competition"].isin({"mexican_primera", 'us_major_league_soccer'}))
index = feats.matches[~ix].index

In [15]:
test_feats = ['acc_cross_nocorner_pct_std_a',
 'acc_cross_nocorner_pct_std_diff',
 'acc_cross_nocorner_pct_std_h',
 'accurate_flick_on_mean_h',
 'accurate_flick_on_mean_a',
 'accurate_freekick_cross_mean_a',
 'accurate_freekick_cross_mean_diff',
 'accurate_freekick_cross_mean_h',
 'accurate_goal_kicks_mean_diff',
 'accurate_goal_kicks_std_a',
 'accurate_goal_kicks_std_diff',
 'accurate_goal_kicks_std_h',
 'accurate_keeper_sweeper_mean_a',
 'accurate_keeper_sweeper_std_diff',
 'accurate_keeper_sweeper_std_h',
 'accurate_pass_std_a',
 'accurate_pass_std_h',
 'accurate_through_ball_mean_diff',
 'att_assist_openplay_std_a',
 'att_assist_openplay_std_h',
 'att_cmiss_left_mean_a',
 'att_cmiss_left_mean_h',
 'att_cmiss_left_std_a',
 'att_cmiss_left_std_diff',
 'att_cmiss_left_std_h',
 'att_goal_high_centre_mean_diff',
 'att_goal_high_centre_std_diff',
 'att_goal_high_right_std_a',
 'att_goal_low_centre_mean_diff',
 'att_goal_low_centre_mean_h',
 'att_goal_low_centre_std_a',
 'att_goal_low_centre_std_h',
 'att_hd_goal_mean_a',
 'att_hd_goal_mean_h',
 'att_hd_target_mean_a',
 'att_hd_target_mean_h',
 'att_ibox_own_goal_mean_diff',
 'att_ibox_own_goal_std_h',
 'att_ibox_own_goal_std_a',
 'att_miss_high_mean_a',
 'att_miss_high_mean_h',
 'att_miss_high_right_mean_diff',
 'att_obxd_right_mean_diff',
 'att_obxd_right_std_diff',
 'att_one_on_one_mean_h',
 'att_one_on_one_mean_a',
 'att_one_on_one_std_diff',
 'att_one_on_one_std_h',
 'att_post_high_std_a',
 'att_post_high_std_h',
 'att_post_right_mean_a',
 'att_post_right_mean_h',
 'attempts_ibox_std_a',
 'attempts_ibox_std_h',
 'backward_pass_mean_diff',
 'backward_pass_mean_h',
 'big_chance_created_mean_h',
 'big_chance_created_std_diff',
 'clean_sheet_std_diff',
 'contentious_decision_mean_a',
 'contentious_decision_mean_diff',
 'contentious_decision_std_a',
 'contentious_decision_std_h',
 'duel_won_pct_mean_diff',
 'effective_clearance_mean_diff',
 'error_lead_to_goal_mean_diff',
 'error_lead_to_goal_std_diff',
 'first_yellow_card_1t_mean_h',
 'first_yellow_card_1t_mean_a',
 'foul_throw_in_std_diff',
 'fouled_final_third_mean_diff',
 'goal_assist_deadball_mean_diff',
 'goal_assist_openplay_std_diff',
 'goal_assist_openplay_std_h',
 'goal_assist_setplay_std_a',
 'goal_assist_setplay_std_diff',
 'goal_assist_setplay_std_h',
 'goal_assist_std_diff',
 'goals_mean_diff',
 'goals_openplay_std_h',
 'goals_openplay_std_a',
 'good_high_claim_mean_diff',
 'high_to_low_goals_mean_a',
 'high_to_low_goals_mean_diff',
 'high_to_low_goals_std_a',
 'high_to_low_goals_std_h',
 'imp_prob_under_goals_0.5_h',
 'imp_prob_under_goals_2.5_diff',
 'interception_mean_a',
 'interception_mean_diff',
 'interception_mean_h',
 'interceptions_in_box_std_a',
 'interceptions_in_box_std_diff',
 'interceptions_in_box_std_h',
 'last_man_tackle_mean_a',
 'last_man_tackle_mean_diff',
 'left_div_right_foot_goals_std_diff',
 'leftside_pass_mean_a',
 'leftside_pass_mean_h',
 'leftside_pass_std_a',
 'long_pass_own_to_opp_mean_diff',
 'long_pass_own_to_opp_mean_h',
 'no_foot_goals_ratio_std_a',
 'no_foot_goals_ratio_std_diff',
 'no_foot_goals_ratio_std_h',
 'odd_ratio_under_corners_10.5',
 'odd_ratio_under_corners_8.5',
 'odds_home_under_both_score_h',
 'odds_away_under_both_score_a',
 'odds_away_under_goals_4.5_a',
 'odds_home_under_goals_4.5_h',
 'odds_away_over_goals_4.5_a',
 'odds_home_over_goals_4.5_h',
 'odds_home_under_goals_0.5_diff',
 'odds_home_under_goals_0.5_h',
 'own_goals_std_h',
 'own_goals_std_a',
 'pen_goals_conceded_mean_diff',
 'penalty_faced_std_a',
 'penalty_won_std_a',
 'penalty_faced_std_h',
 'penalty_won_std_h',
 'poss_won_att_3rd_std_a',
 'poss_won_att_3rd_std_diff',
 'poss_won_att_3rd_std_h',
 'post_scoring_att_std_a',
 'post_scoring_att_std_h',
 'prob_squared_under_goals_2.5',
 'pts_dropped_winning_pos_mean_a',
 'pts_dropped_winning_pos_std_diff',
 'pts_dropped_winning_pos_std_h',
 'ratio_over_goals_2.5_a',
 'ratio_over_goals_2.5_h',
 'ratio_under_goals_2.5_a',
 'ratio_under_goals_2.5_diff',
 'raw_prob_over_goals_1.5',
 'raw_prob_over_goals_2.5',
 'raw_prob_over_goals_3.5',
 'raw_prob_under_goals_1.5',
 'raw_prob_under_goals_2.5',
 'raw_prob_under_goals_3.5',
 'raw_prob_under_goals_4.5',
 'red_card_1t_mean_a',
 'red_card_1t_mean_diff',
 'red_card_1t_mean_h',
 'red_card_2t_mean_a',
 'red_card_2t_mean_diff',
 'red_card_mean_h',
 'red_card_mean_a',
 'red_card_std_diff',
 'right_to_left_goals_mean_diff',
 'right_to_left_goals_std_diff',
 'rightside_pass_div_leftside_pass_mean_a',
 'rightside_pass_div_leftside_pass_mean_h',
 'second_yellow_mean_h',
 'shots_mul_goals_std_h',
 'second_yellow_mean_a',
 'shots_mul_goals_std_a',
 'successful_final_third_passes_mean_a',
 'successful_final_third_passes_mean_diff',
 'successful_final_third_passes_std_a',
 'successful_final_third_passes_std_diff',
 'successful_put_through_std_a',
 'successful_put_through_std_diff',
 'successful_put_through_std_h',
 'total_clearance_mean_a',
 'total_clearance_mean_h',
 'total_fastbreak_std_diff',
 'total_high_claim_mean_a',
 'total_high_claim_mean_diff',
 'total_high_claim_mean_h',
 'total_keeper_sweeper_mean_a',
 'total_keeper_sweeper_mean_h',
 'total_launches_std_h',
 'total_launches_std_a',
 'total_red_card_mean_h',
 'total_red_card_mean_a',
 'total_red_card_std_diff',
 'total_throws_std_a',
 'total_throws_std_diff',
 'total_win_pct_over_goals_1.5_a',
 'total_win_pct_over_goals_1.5_h',
 'total_win_pct_over_goals_2.5_a',
 'total_win_pct_over_goals_2.5_h',
 'total_win_pct_over_goals_3.5_a',
 'total_win_pct_over_goals_3.5_h',
 'total_win_pct_under_goals_1.5_a',
 'total_win_pct_under_goals_1.5_h',
 'total_win_pct_under_goals_2.5_a',
 'total_win_pct_under_goals_2.5_h',
 'total_win_pct_under_goals_3.5_a',
 'total_yel_card_std_a',
 'total_yel_card_std_diff',
 'total_yel_card_std_h',
 'win_pct_away_over_goals_2.5_a',
 'win_pct_away_over_goals_2.5_diff',
 'win_pct_away_under_goals_2.5_a',
 'win_pct_away_under_goals_2.5_diff',
 'win_pct_home_over_goals_2.5_a',
 'win_pct_home_over_goals_2.5_diff',
 'win_pct_home_over_goals_2.5_h',
 'win_pct_home_under_goals_2.5_diff',
 'win_pct_home_under_goals_2.5_h',
 'winner_mean_diff']

In [16]:
def setup_dataset(test_date):
    ds = create_dataset(target=target,
                        test_date=test_date,
                        features=feats,
                        odds_features=True,
                        include_std=True,
                        test_weeks=6,
                        ignore_features=IGNORE_FEATURES,
                        drop_future_matches=False,
                       )
    train_data = ds.train_data.copy()#[ds.train_data.index.map(lambda x: "us_major_league_soccer" not in x and "mexican" not in x)]
    train_data.drop(columns=["hour_rank", "hour_before_16", "is_weekend"], inplace=True)
    train_data = train_data[train_data.index.isin(index)][list(set(test_feats)) + [target]].copy()#.reset_index(drop=True)
    test_data = ds.test_set[ds.test_set.index.isin(index)][list(set(test_feats)) + [target]].copy()#.reset_index(drop=True)
    val_data = ds.val_set[ds.val_set.index.isin(index)].copy()
    setup_kwargs = dict(
        preprocess=True,
        test_data=test_data[train_data.columns.tolist()],#.dropna(),
        #numeric_features=[x for x in train_data.columns.tolist() if x != target],
        #custom_pipeline=loaded,
        #train_size=0.75,
        session_id=123,
        normalize=True,
        normalize_method="robust",
        transformation=True,
        ignore_low_variance=True,
        remove_multicollinearity=False,
        multicollinearity_threshold=0.8,
        n_jobs=-1,
        use_gpu=False,
        profile=False,
        #ignore_features=ignore_features,
        fold_strategy="stratifiedkfold",#"timeseries",
        remove_perfect_collinearity=True,
        create_clusters=False,
        fold=4,
        feature_selection=False,
        # you can use this to keep the 95 % most relevant features (fat_sel_threshold)
        feature_selection_threshold=0.5,
        combine_rare_levels=False,
        rare_level_threshold=0.02,
        pca=False,
        pca_method="linear",
        pca_components=30,
        polynomial_features=False,
        polynomial_degree=2,
        polynomial_threshold=0.05,
        trigonometry_features=False,
        remove_outliers=True,
        outliers_threshold=0.01,
        feature_ratio=False,
        feature_interaction=False,
        # Makes everything slow AF. use to find out possibly interesting features
        interaction_threshold=0.05,
        fix_imbalance=True,
        log_experiment=False,
        verbose=False,
        silent=True,
        experiment_name="lagstest",
    )
    _ = setup(data=train_data, target=target, **setup_kwargs)
    return train_data, test_data, val_data, ds

In [17]:
train_data, test_data, val_data, ds = setup_dataset("4-Dec-2021")

In [18]:
from pycaret.classification import stack_models, ensemble_model, blend_models
def train_ensemble():
    top_models = compare_models(
            n_select=8,
            sort='MCC',
            include=["lr", "lda", "ridge", "et", "rf", "svm"],
            verbose=True,
        )
    tuned_models = [tune_model(model, optimize="MCC", choose_better=True, n_iter=50, search_library="optuna") for model in top_models]
    cali = [calibrate_model(tuned, method="sigmoid", calibrate_fold=4) for tuned in tuned_models]
    blend = blend_models(cali)
    opti = tune_model(blend, optimize="Precision", choose_better=True, n_iter=50, search_library="optuna")
    return opti, cali, tuned_models

In [19]:
from pycaret.classification import stack_models, ensemble_model, blend_models
def train_linear_models():
    top_models = compare_models(
            n_select=8,
            sort='MCC',
            include=["lr", "lda", "ridge", "svm"],
            verbose=True,
        )
    tuned_models = [tune_model(model, optimize="MCC", choose_better=True, n_iter=50, search_library="optuna") for model in top_models]
    cali = [calibrate_model(tuned, method="sigmoid", calibrate_fold=4) for tuned in tuned_models]
    return tuned_models, cali

In [20]:
tuned_linear, cali_linear = train_linear_models()

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6414,0.6893,0.6716,0.6507,0.6610,0.2806,0.2807
1,0.5997,0.6586,0.6321,0.6124,0.6221,0.1969,0.1970
2,0.6216,0.6679,0.6123,0.6442,0.6278,0.2435,0.2439
3,0.6126,0.6475,0.6015,0.6345,0.6175,0.2257,0.2260
Mean,0.6188,0.6658,0.6294,0.6354,0.6321,0.2367,0.2369
SD,0.0152,0.0154,0.0267,0.0145,0.0171,0.0303,0.0303


In [53]:
top_models = compare_models(
            n_select=8,
            sort='MCC',
            include=["lr", "lda", "ridge", "svm"],
            verbose=True,
        )

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.6166,0.6734,0.5836,0.6500,0.6081,0.2351,0.2400,0.3000
lda,Linear Discriminant Analysis,0.6166,0.6705,0.5892,0.6475,0.6115,0.2347,0.2387,0.3025
ridge,Ridge Classifier,0.6166,0.0000,0.5892,0.6476,0.6115,0.2347,0.2387,0.2850
svm,SVM - Linear Kernel,0.5841,0.0000,0.5441,0.6157,0.5734,0.1708,0.1739,0.3050


In [43]:
tuned_linear[3]

SGDClassifier(alpha=0.10126692364605047, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.001022673152073415,
              fit_intercept=False, l1_ratio=0.2203376725525369,
              learning_rate='optimal', loss='hinge', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l1', power_t=0.5,
              random_state=123, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [28]:
evaluate_model(tuned_linear[3])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [51]:
evaluate_model(cali_linear[0])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [55]:
market="goals"
cutoff = 2.5
tips = tips_from_model(model=cali_linear[0],
                test_data=test_data,
                features=feats,
                market=market,
                cutoff=cutoff,
                      )

In [56]:
def show_results(df, groupby=["bet_type", "match_week"]):
    ix = np.logical_and(df["win"]>=0, True)# df["label"]>=1)
    #ix = np.logical_and(df["consensus"] >=0.1, ix)
    ix2 = np.logical_and(df["odds"]>=1.5, df["odds"]<2.)
    ix = np.logical_and(ix, ix2)
    #ix = np.logical_and(ix, ~df["validation"])
    #ix = np.logical_and(ix, df["exp_payoff_prec"] >0.88)
    #ix = np.logical_and(ix, df["exp_payoff"] <1.2)
    #ix = np.logical_and(ix, df["exp_payoff"] >0.8)
    ix = np.logical_and(ix, df["confidence"] >=0.60)
    #ix = np.logical_and(ix, df["confidence"] <=0.75)
    x = df[ix].groupby(groupby)[["win", "profit", "exp_payoff", "exp_payoff_prec", "odds", "confidence"]].mean()
    x["count"] = df[ix].groupby(groupby)[["win"]].count()
    x["buenas"] = df[ix].groupby(groupby)[["win"]].sum().astype(int)
    x["model"] = df["model"].iloc[0]
    return x
show_results(tips)

win    profit  exp_payoff  exp_payoff_prec  \
bet_type        match_week                                                    
goals_over_2.5  2021_19     0.818182  0.434545    1.193888              NaN   
                2021_20     0.700000  0.144000    1.087417              NaN   
                2021_21     0.800000  0.359000    1.130386              NaN   
                2021_22     0.571429  0.040000    1.130597              NaN   
                2021_23     0.500000 -0.140000    1.216788              NaN   
                2021_24     0.625000  0.058750    1.168317              NaN   
                2021_25     0.000000 -1.000000    1.200240              NaN   
goals_under_2.5 2021_19     0.750000  0.255000    1.034973              NaN   
                2021_20     0.800000  0.372000    1.142580              NaN   
                2021_21     0.333333 -0.363333    1.200528              NaN   
                2021_22     1.000000  0.670000    1.025046              NaN   
                2021_23     0.000000 -1.000000    1.011686              NaN   
                2021_24     1.000000  0.755000    1.155859              NaN   

                                odds  confidence  count  buenas  model  
bet_type        match_week                                              
goals_over_2.5  2021_19     1.743636    0.683591     11       9  model  
                2021_20     1.641000    0.664270     10       7  model  
                2021_21     1.685000    0.671600     10       8  model  
                2021_22     1.708571    0.662800      7       4  model  
                2021_23     1.815000    0.667750      4       2  model  
                2021_24     1.730000    0.674975      8       5  model  
                2021_25     1.800000    0.666800      1       0  model  
goals_under_2.5 2021_19     1.630000    0.634150      4       3  model  
                2021_20     1.706000    0.670480      5       4  model  
                2021_21     1.746667    0.686033      3       1  model  
                2021_22     1.670000    0.613800      1       1  model  
                2021_23     1.670000    0.605800      1       0  model  
                2021_24     1.755000    0.662375      4       4  model